In [22]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [23]:
# Creating the state ---> 1st step
class BatsmanState(TypedDict):
    #input attributes
    runs: int
    balls_faced: int
    fours_hit: int
    sixes_hit: int

    #output attributes
    strike_rate: float
    balls_per_boundary: float
    boundary_percentage: float
    summary: str

In [24]:
def calculate_sr(state: BatsmanState):
    sr = (state['runs'] / state['balls_faced']) * 100
    # state['strike_rate'] = sr # No state updation here. Parallel workflow!!!
    return { 'strike_rate': sr } #Partial update of state

In [25]:
def calculate_bpb(state: BatsmanState):
    if (state['fours_hit'] + state['sixes_hit']) == 0:
        # state['balls_per_boundary'] = float('inf') # No state updation here. Parallel workflow!!!
        bpb = float('inf')
        return { 'balls_per_boundary': bpb } #Partial update of state
    else:
        bpb = state['balls_faced'] / (state['fours_hit'] + state['sixes_hit'])
        # state['balls_per_boundary'] = bpb # No state updation here. Parallel workflow!!!
        return { 'balls_per_boundary': bpb } #Partial update of state

In [26]:
def calculate_bp(state: BatsmanState) -> BatsmanState:
    runs_scored_via_boundaries = ((state['fours_hit'] * 4) + (state['sixes_hit'] * 6))
    bp = (runs_scored_via_boundaries / state['runs']) * 100
    # state['boundary_percentage'] = bp # No state updation here. Parallel workflow!!!
    return {'boundary_percentage': bp } #Partial update of state

In [27]:
def summary(state: BatsmanState) -> BatsmanState:
    summary = f"""
    Runs: {state['runs']}
    Balls Faced: {state['balls_faced']}
    Fours Hit: {state['fours_hit']}
    Sixes Hit: {state['sixes_hit']}
    Strike Rate: {state['strike_rate']}
    Balls Per Boundary: {state['balls_per_boundary']}
    Boundary Percentage: {state['boundary_percentage']}
"""
    # state['summary'] = summary # No state updation here. Parallel workflow!!!
    return {'summary': summary } #Partial update of state

In [28]:
# Resgistering the graph... 2nd step
graph = StateGraph(BatsmanState)

# Adding nodes to the graph...
graph.add_node('calculate_strike_rate', calculate_sr)
graph.add_node('calculate_balls_per_boundary', calculate_bpb)
graph.add_node('calculate_boundary_percentage', calculate_bp)
graph.add_node('summary', summary)

# Adding edges to the graph... 4th step
graph.add_edge(START, 'calculate_strike_rate')
graph.add_edge(START, 'calculate_balls_per_boundary')
graph.add_edge(START, 'calculate_boundary_percentage')

# Now connecting all the node to the summary node
graph.add_edge('calculate_strike_rate', 'summary')
graph.add_edge('calculate_balls_per_boundary', 'summary')
graph.add_edge('calculate_boundary_percentage', 'summary')

# Finally connecting summary to END
graph.add_edge('summary', END)

workflow = graph.compile()

In [14]:
initial_state: BatsmanState = {
    'runs': 120,
    'balls_faced': 100,
    'fours_hit': 10,
    'sixes_hit': 5,
}

In [29]:
final_state = workflow.invoke(initial_state)

In [31]:
final_state

{'runs': 120,
 'balls_faced': 100,
 'fours_hit': 10,
 'sixes_hit': 5,
 'strike_rate': 120.0,
 'balls_per_boundary': 6.666666666666667,
 'boundary_percentage': 58.333333333333336,
 'summary': '\n    Runs: 120\n    Balls Faced: 100\n    Fours Hit: 10\n    Sixes Hit: 5\n    Strike Rate: 120.0\n    Balls Per Boundary: 6.666666666666667\n    Boundary Percentage: 58.333333333333336\n'}